In [1]:
import numpy as np
import random
from collections import deque
import warnings
import numpy as np
import math

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from mdesc.models import (Eval, Sensitivity)
from mdesc.data.datamanager import DataUtilities as DU

In [2]:
# initialize plotly notebook_mode
init_notebook_mode(connected=True)

In [3]:
import os
os.getcwd()

'/Users/jasonlewris/Desktop/GitProjects/model-describer'

In [30]:
# this is a test file
# sensitivity plot creation and testing

modelObjc = RandomForestRegressor()

# wine quality dataset example
wine = pd.read_csv('/Users/jasonlewris/Desktop/GitProjects/model-describer/debug/data/wine.csv')

groupby_df = wine[['alcohol', 'Type']]
groupby_df['all'] = 1

ydepend = 'quality'

X = wine.loc[:, wine.columns != ydepend]
y = wine.loc[:, ydepend]

# y = np.array(['bad' if val < 5 else 'good' for val in y])

X = pd.get_dummies(X)

p_arr, percentile_list, str_p_list = DU.create_percentiles(X['free sulfur dioxide'])
pd.DataFrame(p_arr, columns=['value'])
modelObjc.fit(X, y)

RE = Sensitivity(prediction_fn=modelObjc.predict,
                      model_type='regression')
res = RE.fit_transform(X=X, y=y, groupby_df=groupby_df)


/Users/jasonlewris/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



0 alcohol
high


/Users/jasonlewris/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1018: RuntimeWarning:

Mean of empty slice



low
medium
1 Type
Red
White
2 all
1


In [31]:
res.head()

,MSE,Total,dtype,errNeg,errPos,groupByValue,groupByVarName,predictedYSmooth,std_change,x_name,x_value
0,0.0000,1.0,Continuous,0.00,0.00,high,alcohol,0.0,0.1646,volatile acidity,0.115
0,0.5775,4.0,Continuous,0.00,0.70,high,alcohol,0.7,0.1646,volatile acidity,0.160
0,0.1000,6.0,Continuous,-0.25,0.30,high,alcohol,-0.1,0.1646,volatile acidity,0.170
0,0.1500,6.0,Continuous,-0.20,0.25,high,alcohol,0.1,0.1646,volatile acidity,0.180
0,0.2067,6.0,Continuous,-0.20,0.30,high,alcohol,0.2,0.1646,volatile acidity,0.190


In [40]:
GROUPBY_NAME = 'alcohol'
X_NAME = 'volatile acidity'
CHART_TYPE = 'sensitivity'
# v_acid = res.loc[(res['groupByVarName'] == GROUPBY_NAME)&(res['x_name'] == X_NAME)]

In [41]:
rgb = lambda: random.randint(0,255)

In [42]:
import random



rgb = lambda: random.randint(0,255)


def create_error_trace(level, x, x_rev, y, y_upper, y_lower, 
                rgb=None):
    # pick colors
    if rgb is None:
        r, g, b = rgb(), rgb(), rgb()
    else:
        r, g, b = rgb
    trace1 = go.Scatter(
        x=x+x_rev,
        y=y_upper + y_lower,
        fill='tozerox',
        fillcolor='rgba({},{},{},0.2)'.format(r, g, b),
        line=dict(color='rgba(255,255,255,0)'),
        showlegend=False,
        name=level,
    )

    trace2 = go.Scatter(
        x=x,
        y=y,
        line=dict(color='rgb({},{},{})'.format(r, g, b)),
        mode='lines',
        name=level,
    )
    return (trace1, trace2)

def create_sensitivity_trace(level, x, y, rgb=None):
    # pick colors
    if rgb is None:
        r, g, b = rgb(), rgb(), rgb()
    else:
        r, g, b = rgb
    trace1 = go.Scatter(
        x=x,
        y=y,
        line=dict(color='rgb({},{},{})'.format(r, g, b)),
        mode='lines',
        name=level,
    )
    return trace1

def create_nested_error_traces(df=None, 
                        groupby_name=None):

    traces, all_indices, all_x_names = [], [], []
    col_lookup, layout_lookup, range_lookup, title_lookup = {}, {}, {}, {}
    
    # iterate over levels within groupby_name
    levels = df.loc[df['groupByVarName'] == groupby_name, 'groupByValue'].unique()
    rgb_anchor = [(rgb(), rgb(), rgb()) for i in levels]

    for x_idx, X_NAME in enumerate(df['x_name'].unique()):
        all_x_names.append(X_NAME)
        col_lookup[X_NAME] = []

        v_acid = df.loc[(df['groupByVarName'] == groupby_name)&(df['x_name'] == X_NAME)]
        master_x = v_acid['x_value'].values
        range_lookup[X_NAME] = [np.min(master_x), np.max(master_x)]
        title_lookup[X_NAME] = '{} for {} error chart'.format(X_NAME, groupby_name)

        for l_idx, level in enumerate(levels):
            mask = v_acid['groupByValue'] == level
            y = v_acid.loc[mask, 'predictedYSmooth'].values
            y_lower = (y + v_acid.loc[mask, 'errNeg'].values).tolist()
            y_lower = y_lower[::-1]
            y_upper = (y + v_acid.loc[mask, 'errPos'].values).tolist()
            x = v_acid.loc[mask, 'x_value'].tolist()
            x_rev = x[::-1]

            trace1, trace2 = create_error_trace(level, x, x_rev, y, y_upper, y_lower, 
                                                rgb=rgb_anchor[l_idx])
            trace_tracker_1 = len(traces)
            trace_tracker_2 = trace_tracker_1 + 1
            traces.append(trace1)
            all_indices.append(trace_tracker_1)
            traces.append(trace2)
            all_indices.append(trace_tracker_2)
            col_lookup[X_NAME].append(trace_tracker_1)
            col_lookup[X_NAME].append(trace_tracker_2)
            
    return (traces, all_indices, all_x_names, col_lookup, layout_lookup, range_lookup, title_lookup)

def create_nested_sensitivity_traces(df=None, 
                                     groupby_name=None):

    traces, all_indices, all_x_names = [], [], []
    col_lookup, layout_lookup, range_lookup, title_lookup = {}, {}, {}, {}
    
    # iterate over levels within groupby_name
    levels = df.loc[df['groupByVarName'] == groupby_name, 'groupByValue'].unique()
    rgb_anchor = [(rgb(), rgb(), rgb()) for i in levels]


    for x_idx, X_NAME in enumerate(df['x_name'].unique()):
        all_x_names.append(X_NAME)
        col_lookup[X_NAME] = []

        v_acid = df.loc[(df['groupByVarName'] == groupby_name)&(df['x_name'] == X_NAME)]
        master_x = v_acid['x_value'].values
        range_lookup[X_NAME] = [np.min(master_x), np.max(master_x)]
        title_lookup[X_NAME] = '{} for {} sensitivity chart'.format(X_NAME, groupby_name)

        for l_idx, level in enumerate(levels):
            mask = v_acid['groupByValue'] == level
            y = v_acid.loc[mask, 'predictedYSmooth'].values
            x = v_acid.loc[mask, 'x_value'].tolist()

            trace1 = create_sensitivity_trace(level, x, y, rgb=rgb_anchor[l_idx])
            trace_tracker_1 = len(traces)
            traces.append(trace1)
            all_indices.append(trace_tracker_1)
            col_lookup[X_NAME].append(trace_tracker_1)
            
    return (traces, all_indices, all_x_names, col_lookup, layout_lookup, range_lookup, title_lookup)
            
            
GROUPBY_NAME = 'alcohol'
X_NAME = 'volatile acidity'
Y_NAME = 'Quality'

if CHART_TYPE == 'sensitivity':

    results = create_nested_sensitivity_traces(df=res, groupby_name=GROUPBY_NAME)
else:
    results = create_nested_error_traces(df=res, groupby_name=GROUPBY_NAME)

traces, all_indices, all_x_names, col_lookup, layout_lookup, range_lookup, title_lookup = results

In [43]:
def create_buttons(all_x_names, col_lookup=None, 
                   groupby_name=None, range_lookup=None, 
                  all_indices=None, chart_type='error'):

    buttons = []

    for x_name in all_x_names:
        format_dict = {}
        format_dict['label'] = x_name
        format_dict['method'] = 'update'
        format_dict['args'] = [
            {'visible': np.isin(all_indices, col_lookup[x_name])}, 
            {'title': '{} VIZ {}; Groupby: {}'.format(chart_type, x_name, groupby_name), 
            'xaxis': dict(
                gridcolor='rgb(255,255,255)',
                range=range_lookup[x_name],
                showgrid=True,
                showline=False,
                showticklabels=True,
                tickcolor='rgb(127,127,127)',
                ticks='outside',
                zeroline=False,
                title=x_name
            )}
        ]
        buttons.append(format_dict)
        
    return buttons


def create_update_menu():
    
    updatemenus = list([
        dict(active=-1, 
            buttons=buttons, 
            direction = 'down',
                pad = {'r': 10, 't': 10},
                showactive = True,
                xanchor = 'left',
                yanchor = 'top', 
            y=1.12, 
            x=1, 
            )
    ])
    
    return updatemenus


buttons = create_buttons(all_x_names, col_lookup=col_lookup,
                        groupby_name=GROUPBY_NAME, range_lookup=range_lookup,
                        all_indices=all_indices, chart_type=CHART_TYPE)

updatemenus = create_update_menu()


In [44]:
buttons

[{'args': [{'visible': array([ True,  True,  True, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False])},
   {'title': 'sensitivity VIZ volatile acidity; Groupby: alcohol',
    'xaxis': {'gridcolor': 'rgb(255,255,255)',
     'range': [0.08, 0.885],
     'showgrid': True,
     'showline': False,
     'showticklabels': True,
     'tickcolor': 'rgb(127,127,127)',
     'ticks': 'outside',
     'title': 'volatile acidity',
     'zeroline': False}}],
  'label': 'volatile acidity',
  'method': 'update'},
 {'args': [{'visible': array([False, False, False,  True,  True,  True, False, False, False,
           False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False])},
   {'title': 'sensitivity VIZ citric acid; Groupby: alcohol',
    'xaxis': {'gridcolor': 'rgb(255,255,255)',
     '

In [47]:
annotations = list([dict(text='Feature Name', 
                  showarrow=False, 
                  align='left', 
                  yref='paper', 
                        x=3.6, 
                        y=1)])

layout = go.Layout(
        title='{} Visualization Chart'.format(CHART_TYPE),
        paper_bgcolor='rgb(255,255,255)',
        plot_bgcolor='rgb(229,229,229)',
        xaxis=dict(
            gridcolor='rgb(255,255,255)',
            #range=[np.min(x), np.max(x)],
            showgrid=True,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(127,127,127)',
            ticks='outside',
            zeroline=False,
            title='x axis'
        ),
        yaxis=dict(
            gridcolor='rgb(255,255,255)',
            showgrid=True,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(127,127,127)',
            ticks='outside',
            zeroline=False, 
            title=Y_NAME
        ),
        updatemenus=updatemenus,
    )

layout['updatemenus'] = updatemenus
# layout['annotations'] = annotations

In [48]:
data = traces
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
class Class1(object):
    
    def __init__(self, var1=None):
        self.var1 = var1
        
    def do_something(self):
        print(self.var1)
        if isinstance(self, Class2):
            print('we are in class 2, lets not celebrate')
        if isinstance(self, Class1):
            print('we are in class 1, lets celebrate')
        
        
class Class2(Class1):
    
    def __init__(self, var1=None, var2=10):
        
        self.var2 = var2
        super(Class2, self).__init__(var1=var1)
        
    
c = Class1(var1=10)
c.do_something()

In [ ]:
c = Class2(var1=100, var2=20)
c.do_something()

In [ ]:
dir(c)

In [ ]:
str(c.__class__).split('.')

In [ ]:
type(c.__class__)

In [ ]:
isinstance(c, Class2)

In [ ]:
type(c.)